## Importing the LonelyBoy Library (github.com/insert-generic-name-here/lonelyboy)

In [1]:
import os, sys
sys.path.append(os.path.join(os.path.expanduser('~'), 'Documents/Coding/Python/'))
# sys.path

from lonelyboy.geospatial import plots as gsplt
from lonelyboy.geospatial import preprocessing as gspp
from lonelyboy.timeseries import lbtimeseries as tspp
from lonelyboy.geospatial import group_patterns_v2 as gsgp## Importing the LonelyBoy Library (github.com/insert-generic-name-here/lonelyboy)

## Importing all other Essential Libraries
#### (DO NOT FORGET TO EXECUTE THE FUNCTIONS IN THE BOTTOM CELLS)

In [2]:
import psycopg2
import numpy as np
import configparser
import pandas as pd
import geopandas as gpd
import contextily as ctx
from random import choice
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.cluster import DBSCAN, KMeans, MeanShift
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from shapely.geometry import Point, LineString, shape
from haversine import haversine
from datetime import datetime, timedelta

In [3]:
from multiprocessing import cpu_count, Pool
from functools import partial
import datetime

## Import Libraries for Visualizations

In [4]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

import PyQt5
import matplotlib.pyplot as plt
from matplotlib import style;  style.use('ggplot')
style.use('ggplot')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

get_ipython().magic('matplotlib qt')
# get_ipython().magic('matplotlib inline')

## Get & Plot a Random Vessel's Trajectory

In [5]:
%%time
import random

mmsis = pd.read_pickle('data/pkl/mmsi_list.pckl')
mmsi_selected = random.choice(random.choice(mmsis))

csv_iter = gspp.read_csv_generator('data/csv/nari_dynamic.csv')
chunk = pd.concat((chunk[chunk['mmsi'] == 304010909] for chunk in csv_iter), ignore_index=True)
chunk = gspp.gdf_from_df(chunk, crs={'init':'epsg:4326'})

print (mmsi_selected)
chunk.head()

273430590
Wall time: 21.1 s


In [6]:
chunk.head()

,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom
0,304010909,0.0,-126.0,15.2,22.7,26,-6.435827,47.690422,1446441722,POINT (-6.435827 47.690422)
1,304010909,0.0,0.0,15.1,25.7,26,-6.433708,47.693592,1446441766,POINT (-6.433708 47.693592)
2,304010909,0.0,0.0,15.2,25.9,25,-6.432607,47.695168,1446441792,POINT (-6.4326067 47.695168)
3,304010909,0.0,0.0,15.1,23.8,26,-6.432422,47.695490,1446441800,POINT (-6.432421700000001 47.69549)
4,304010909,0.0,0.0,102.3,23.9,26,-6.430683,47.698040,1446441840,POINT (-6.430683 47.69804)


# TEST: Fetching and Plotting a Trajectory along with outlier points

In [44]:
ports = pd.read_pickle('data/pkl/ports_raw.pkl')
ports = gpd.GeoDataFrame(ports, crs={'init':'epsg:3857'}, geometry='geom')

chunk2 = gspp.calculate_velocity(chunk.sort_values('ts'))
chunk2 = chunk2.loc[chunk2.velocity >= 120]

df1 = ports
df2 = chunk
df3 = chunk2
figsize=(10, 10)
color=['steelblue', 'red', 'black']


df1.crs = {'init': 'epsg:4326'}
ax = df1.to_crs(epsg=3857).plot(figsize=figsize, color=color[0])
if df2 is not None:
    df2.crs = {'init': 'epsg:4326'}
    df2.to_crs(epsg=3857).plot(figsize=figsize, color=color[1], ax=ax)
if df3 is not None:
    df3.crs = {'init': 'epsg:4326'}
    df3.to_crs(epsg=3857).plot(figsize=figsize, color=color[2], ax=ax)
       

ctx.add_basemap(ax, attribution='')
ax.margins(0)
ax.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
plt.show()

C:\Users\This PC\.conda\envs\gis\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\This PC\.conda\envs\gis\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\This PC\.conda\envs\gis\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/p

In [43]:
chunk3 = chunk2.sort_values('ts')

chunk3['current_loc'] = chunk3.geom.apply(lambda x: (x.x,x.y))
chunk3['next_loc'] = chunk3.geom.shift(-1)
chunk3 = chunk3[:-1]
chunk3.loc[:,'next_loc'] = chunk3.next_loc.apply(lambda x : (x.x,x.y))
# get the distance traveled in n-miles and multiply by the rate given (3600/secs for knots)
chunk3.loc[:,'pair_distance'] = chunk3[['current_loc', 'next_loc']].apply(lambda x : gspp.haversine(x[0], x[1]), axis=1)
chunk3.drop(['current_loc', 'next_loc'], axis=1, inplace=True)

chunk3.loc[:,'pair_dt'] = np.abs(chunk2.sort_values('ts').ts.diff(-1).dropna())

chunk3

,mmsi,status,turn,speed,course,heading,lon,lat,ts,geom,velocity,pair_distance,pair_dt
61,304010909,0.0,0.0,15.7,26.6,29,-6.392515,47.752106,1446442677,POINT (-6.392515 47.752106),3.972057e+02,1.021701,12.0
63,304010909,0.0,0.0,15.3,24.8,29,-6.383332,47.752422,1446442689,POINT (-6.383332 47.752422),3.059797e+02,7.454323,938.0
192,304010909,0.0,0.0,14.7,27.0,29,-6.349515,47.810665,1446443627,POINT (-6.349515 47.810665),1.369021e+06,704.284993,15.0
200,304010909,0.0,-126.0,14.6,26.5,29,-0.015737,47.811270,1446443642,POINT (-0.015736667 47.81127),4.562597e+05,703.903361,48.0
138,304010909,0.0,0.0,14.9,27.5,30,-6.346082,47.815228,1446443690,POINT (-6.3460817 47.815228),3.266309e+05,672.133755,14.0


# TEST: New Resampling Method

In [37]:
chunk_resampled = gspp.resample_geospatial(chunk, features=['mmsi', 'status', 'turn', 'speed', 'course', 'heading', 'lon', 'lat', 'ts'], rule=f'{1*60}S')
gsplt.map_plot(chunk_resampled, column='ts', figsize=(10,10), legend=True)

In [38]:
chunk_resampled.head()

,mmsi,status,turn,speed,course,heading,lon,lat,ts,datetime,geom
0,249959000.0,2.0,127.0,2.399305,129.713647,183.999131,-5.585573,48.013347,1.455944e+09,2016-02-20 04:59:00,POINT (-5.585573375994205 48.01334688272365)
1,249959000.0,2.0,127.0,2.395828,129.781883,183.994784,-5.585080,48.013021,1.455944e+09,2016-02-20 05:00:00,POINT (-5.58508025596523 48.0130212963419)
2,249959000.0,2.0,127.0,2.392351,129.850120,183.990438,-5.584587,48.012696,1.455944e+09,2016-02-20 05:01:00,POINT (-5.584587135936255 48.01269570996016)
3,249959000.0,2.0,127.0,2.388874,129.918356,183.986092,-5.584094,48.012370,1.455945e+09,2016-02-20 05:02:00,POINT (-5.584094015907279 48.01237012357841)
4,249959000.0,2.0,127.0,2.385397,129.986592,183.981746,-5.583601,48.012045,1.455945e+09,2016-02-20 05:03:00,POINT (-5.583600895878305 48.01204453719667)


## Test Plots

In [9]:
csv_iter = gspp.read_csv_generator('data/csv/mmsi/205067000.csv')
chunk = pd.concat((chunk[chunk['mmsi'] == 205067000] for chunk in csv_iter), ignore_index=True)
chunk = gspp.gdf_from_df(chunk, crs={'init':'epsg:4326'})

ports = pd.read_pickle('data/pkl/ports_raw.pkl')
ports = gspp.create_port_bounds(ports, port_radius=2000)

In [14]:
gsplt.map_plot(chunk, ports, column='ts', color=[None, 'r'], figsize=(10,10), legend=True)

In [28]:
tmp = chunk.ts.diff()/(60*60*24)
plt.title('Trajectory Time Delta Plot')
plt.xlabel('Record Index')
plt.ylabel('Time Delta (in Days)')
tmp.plot()

## Fetch Data for a Certain Period

In [6]:
%%time
import random

mmsis = pd.read_pickle('data/pkl/mmsi_list.pckl')
mmsi_selected = random.choice(random.choice(mmsis))

csv_iter = gspp.read_csv_generator('data/csv/nari_dynamic.csv')
chunk = pd.concat((chunk[(chunk['ts'] >= 1456790400) & (chunk['ts'] <= 1456876800)] for chunk in csv_iter), ignore_index=True)
chunk = gspp.gdf_from_df(chunk, crs={'init':'epsg:4326'})

print (mmsi_selected)
chunk.head()

245257000
Wall time: 31 s


In [7]:
ports = pd.read_pickle('data/pkl/ports_raw.pkl')
ports.head()

,gid,gml_id,por_id,libelle_po,insee_comm,por_x,por_y,geom
0,1,port.1,1,Le Vivier-sur-Mer,35361,297025.0,2408370.0,POINT (-1.771798868659233 48.60274269672541)
1,2,port.10,10,Saint-Samson sur Rance,22327,279335.0,2396060.0,POINT (-2.001990119062326 48.48369993456267)
2,3,port.100,100,Douarnenez,29046,103135.0,2365330.0,POINT (-4.341204251638414 48.09709590770091)
3,4,port.101,101,Brézellec,29028,79105.4,2364190.0,POINT (-4.661115947908725 48.06804110561076)
4,5,port.102,102,Sein,29083,64562.5,2362180.0,POINT (-4.852944548180974 48.03825273921113)


In [8]:
PLT_FIG_WIDTH = 3.787
PLT_FIG_HEIGHT = PLT_FIG_WIDTH / 1.618

gsplt.map_plot(chunk, ports, figsize=(10, 10), title='', color=['navy', 'orangered'])
plt.axis('off')
# plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', 'Dataset_Snapshot.pdf'), dpi=500, bbox_inches='tight', pad_inches=0)
plt.savefig(os.path.join('.', 'experimental_figures', 'figs_final', 'Dataset_Snapshot.png'), dpi=500, bbox_inches='tight', pad_inches=0)